In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import bisect
import json
from datetime import date

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path = replace_slash('input_data\\' + 'armitz/miner/ARX_PERF.xlsx')
fes_path = replace_slash('input_data\\armitz/miner/QINF.xlsx')

In [5]:
perf_df = perf_reader(perf_path)
fes_df = fes_reader(fes_path)
SOIL_CUT = 60

started reading perf xl
done reading perf xl
started reading fes xl
done reading fes xl


In [6]:
fes_df

,well,top,bot,soil
0,1,NaN,1648.4,NaN
1,1,1669.6,1670.6,69.5
2,1,1678.4,1679.2,NaN
3,1,1683.0,1684.0,NaN
4,1,1686.0,1687.0,NaN
...,...,...,...,...
156269,32960,1901.0,1901.8,52.9
156270,32960,1903.0,1905.0,NaN
156271,32960,1914.0,1915.4,NaN
156272,32960,1917.6,1919.0,NaN


In [16]:
def get_actual_perf(perf_df):
    act_perf = []
    for well in tqdm(perf_df['well'].unique()):
        well_df = perf_df[perf_df['well'] == well]
        act_perf_well = []
        for date in well_df['date'].unique():
            date_df = well_df[well_df['date'] == date]
            for top, bot, perf_type in date_df[['top', 'bot', 'type']].values:
                idx = bisect.bisect_left([t['bot'] for t in act_perf_well], top)
                if idx == len(act_perf_well):
                    act_perf_well.append({'well': well, 'top': top,'bot': bot, 'perf_type': perf_type})
                else:
                    if act_perf_well[idx]['top'] > top:
                        if act_perf_well[idx]['perf_type'] == perf_type:
                            act_perf_well[idx]['top'] = top
                        else:
                            act_perf_well.insert(idx, {'well': well, 'top': top,'bot': bot if act_perf_well[idx]['top'] > bot else act_perf_well[idx]['top'],
                                                   'perf_type': perf_type})
                    if act_perf_well[idx]['bot'] < bot:
                        if act_perf_well[idx]['perf_type'] == perf_type:
                            act_perf_well[idx]['bot'] = bot
                        else:
                            act_perf_well.insert(idx + 1, {'well': well, 'top': act_perf_well[idx]['bot'],'bot': bot, 'perf_type': perf_type})

#         act_perf[well] = act_perf_well
        act_perf.extend(act_perf_well)
    return act_perf


#

perf =  get_actual_perf(perf_df)

100%|██████████| 4093/4093 [00:45<00:00, 89.64it/s] 


In [6]:
perf_df

,well,type,date,top,bot
48492,9642д,1,1991-03-11 00:00:00,1316.0,1340.5
48491,9642д,0,1991-03-10 00:00:00,1676.8,1679.8
48490,9642д,1,1982-01-16 00:00:00,1676.8,1679.8
54866,9636,1,2003-09-10 23:00:00,1736.0,1738.0
54867,9636,1,2003-09-10 23:00:00,1739.7,1740.9
...,...,...,...,...,...
24139,12,1,1991-04-18 23:00:00,10.5,36.0
24131,11,1,1991-04-11 23:00:00,67.5,69.0
24132,11,1,1991-04-11 23:00:00,69.0,70.5
24130,9,0,2007-09-28 23:00:00,37.0,42.0


In [10]:
# проверка пласта на перфорированность
def is_perf(top, bot, ints):
    if ints is None:
        return False
    for row in ints.itertuples():
        if (top < row.bot) and (row.top < bot):
            return True
    return False


def find_layers(perf_df, fes_df, soil_cut):
    lost_layers = pd.DataFrame(columns=['well', 'top', 'bot', 'soil', 'is_perf'])
    for well in tqdm(fes_df['well'].unique()):
        well_df = fes_df[fes_df['well'] == well][['well', 'top', 'bot', 'soil']]
        well_df.dropna(inplace=True)
        if len(well_df) == 0:
            continue
        ints = perf_df[perf_df['well'] == well][['top', 'bot']]
        well_df['is_perf'] = well_df\
            .apply(lambda x: is_perf(x['top'], x['bot'], ints), axis=1)
        non_perf = well_df[(~well_df['is_perf']) &
                           (well_df['soil'] > soil_cut)]
        lost_layers = lost_layers.append(non_perf, ignore_index=True)
    return lost_layers

In [11]:
plasts = find_layers(perf_df, fes_df, SOIL_CUT)

  0%|          | 22/6457 [00:00<01:31, 69.99it/s]

Pandas(Index=43709, top=1593.0, bot=1595.0)
Pandas(Index=43709, top=1593.0, bot=1595.0)
Pandas(Index=43710, top=1606.5, bot=1609.5)
Pandas(Index=43711, top=1619.2, bot=1621.6)
Pandas(Index=43712, top=1631.2, bot=1633.0)
Pandas(Index=43715, top=1593.5, bot=1596.8)
Pandas(Index=43716, top=1605.0, bot=1606.5)
Pandas(Index=43714, top=1605.0, bot=1606.5)
Pandas(Index=43713, top=1593.5, bot=1596.8)


  1%|          | 37/6457 [00:00<01:43, 61.78it/s]

Pandas(Index=43750, top=1607.0, bot=1610.0)
Pandas(Index=43750, top=1607.0, bot=1610.0)
Pandas(Index=43751, top=1617.0, bot=1620.8)
Pandas(Index=43750, top=1607.0, bot=1610.0)
Pandas(Index=43751, top=1617.0, bot=1620.8)
Pandas(Index=43752, top=1620.8, bot=1621.0)
Pandas(Index=43747, top=1607.0, bot=1610.0)
Pandas(Index=43748, top=1617.0, bot=1620.8)
Pandas(Index=43749, top=1620.8, bot=1621.0)
Pandas(Index=43745, top=1607.0, bot=1610.0)
Pandas(Index=43746, top=1618.4, bot=1620.8)
Pandas(Index=43743, top=1607.0, bot=1610.0)
Pandas(Index=43744, top=1618.4, bot=1620.8)
Pandas(Index=43741, top=1607.5, bot=1609.5)
Pandas(Index=43742, top=1619.0, bot=1620.0)
Pandas(Index=43739, top=1607.0, bot=1610.0)
Pandas(Index=43740, top=1618.4, bot=1620.8)
Pandas(Index=43737, top=1607.0, bot=1610.0)
Pandas(Index=43738, top=1618.4, bot=1620.8)
Pandas(Index=43735, top=1607.0, bot=1610.0)
Pandas(Index=43736, top=1618.4, bot=1620.8)
Pandas(Index=43733, top=1607.0, bot=1610.0)
Pandas(Index=43734, top=1618.4, 

  1%|          | 62/6457 [00:00<01:29, 71.46it/s]

...

Pandas(Index=54751, top=1787.0, bot=1790.0)
Pandas(Index=54747, top=1787.0, bot=1790.0)
Pandas(Index=54748, top=1787.0, bot=1790.0)
Pandas(Index=54749, top=1807.0, bot=1808.0)
Pandas(Index=54750, top=1812.6, bot=1813.6)
Pandas(Index=54746, top=1819.2, bot=1820.4)
Pandas(Index=54745, top=1819.2, bot=1820.4)
Pandas(Index=54741, top=1823.4, bot=1824.2)
Pandas(Index=54742, top=1824.2, bot=1826.0)
Pandas(Index=54743, top=1826.0, bot=1828.0)
Pandas(Index=54744, top=1828.0, bot=1830.4)
Pandas(Index=54737, top=1823.4, bot=1824.2)
Pandas(Index=54738, top=1824.2, bot=1825.4)
Pandas(Index=54739, top=1827.9, bot=1828.0)
Pandas(Index=54740, top=1828.0, bot=1830.4)
Pandas(Index=54751, top=1787.0, bot=1790.0)
Pandas(Index=54747, top=1787.0, bot=1790.0)
Pandas(Index=54748, top=1787.0, bot=1790.0)
Pandas(Index=54749, top=1807.0, bot=1808.0)
Pandas(Index=54750, top=1812.6, bot=1813.6)
Pandas(Index=54746, top=1819.2, bot=1820.4)
Pandas(Index=54745, top=1819.2, bot=1820.4)
Pandas(Index=54741, top=1823.4, 

  9%|▉         | 597/6457 [00:11<01:50, 53.08it/s]

Pandas(Index=37435, top=1777.0, bot=1778.8)
Pandas(Index=37436, top=1778.8, bot=1779.2)
Pandas(Index=37437, top=1779.2, bot=1780.0)
Pandas(Index=37434, top=1787.9, bot=1788.9)
Pandas(Index=37433, top=1787.9, bot=1788.9)
Pandas(Index=37435, top=1777.0, bot=1778.8)
Pandas(Index=37435, top=1777.0, bot=1778.8)
Pandas(Index=37436, top=1778.8, bot=1779.2)
Pandas(Index=37435, top=1777.0, bot=1778.8)
Pandas(Index=37436, top=1778.8, bot=1779.2)
Pandas(Index=37437, top=1779.2, bot=1780.0)
Pandas(Index=37434, top=1787.9, bot=1788.9)
Pandas(Index=37435, top=1777.0, bot=1778.8)
Pandas(Index=37436, top=1778.8, bot=1779.2)
Pandas(Index=37437, top=1779.2, bot=1780.0)
Pandas(Index=37434, top=1787.9, bot=1788.9)
Pandas(Index=37433, top=1787.9, bot=1788.9)
Pandas(Index=37460, top=1673.7, bot=1675.2)
Pandas(Index=37459, top=1673.7, bot=1675.2)
Pandas(Index=37457, top=1682.4, bot=1685.2)
Pandas(Index=37458, top=1696.5, bot=1697.6)
Pandas(Index=37456, top=1682.4, bot=1685.2)
Pandas(Index=37455, top=1696.5, 

  9%|▉         | 607/6457 [00:11<01:50, 52.87it/s]


KeyboardInterrupt: 

In [7]:
plasts


,well,top,bot,soil,is_perf
0,1,1669.6,1670.6,69.5,False
1,1,1689.6,1692.4,91.7,False
2,1,1699.6,1702.8,91.0,False
3,1,1709.8,1712.0,77.0,False
4,1н,1583.6,1586.4,70.0,False
...,...,...,...,...,...
25660,32957,1803.5,1804.6,66.0,False
25661,32958,1653.0,1653.7,68.8,False
25662,32958,1654.0,1654.7,71.7,False
25663,32959,2059.4,2060.2,63.9,False
